### MaskRCNN training. Balloon dataset

In [1]:
import os
os.chdir('../src')
import tensorflow as tf

from samples.balloon import balloon
from preprocess import preprocess
from preprocess import augmentation as aug
from training import train_model
from model import mask_rcnn_functional
from common.utils import tf_limit_gpu_memory

In [2]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2021-09-25T15:50:01.208992+03:00

Python implementation: CPython
Python version       : 3.7.7
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.4.0-65-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

tensorflow: 2.2.0



In [3]:
tf_limit_gpu_memory(tf, 4500)

1 Physical GPUs, 1 Logical GPUs Memory limit: 4500
Physical GPU-devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
from common.config import CONFIG

In [5]:
CONFIG.update({'class_dict': {'balloon': 1, 'background': 0},
               'num_classes': 2,
               'epochs': 30,
              },
             )
CONFIG.update({'meta_shape': (1 + 3 + 3 + 4 + 1 + CONFIG['num_classes']),})
CONFIG

{'image_shape': (512, 512, 3),
 'img_size': 512,
 'backbone': 'mobilenet',
 'meta_shape': 14,
 'num_classes': 2,
 'class_dict': {'balloon': 1, 'background': 0},
 'normalization': {'mean': [0.485, 0.456, 0.406],
  'std': [0.229, 0.224, 0.225]},
 'image_min_dim': 300,
 'image_min_scale': 0,
 'image_max_dim': 512,
 'image_resize_mode': 'square',
 'use_mini_masks': False,
 'mini_mask_shape': (32, 32),
 'mask_shape': (28, 28),
 'epochs': 30,
 'gpu_num': 1,
 'batch_size': 1,
 'images_per_gpu': 1,
 'training': True,
 'log_per_steps': 5,
 'use_multiprocessing': True,
 'workers': 6,
 'callback': {'checkpoints_dir': '../logs/scalars',
  'reduce_lr_on_plateau': 0.98,
  'reduce_lr_on_plateau_patience': 10,
  'save_weights_only': True,
  'save_best_only': True,
  'histogram_freq': 0,
  'profile_batch': '1,2'},
 'backbone_strides': [4, 8, 16, 32, 64],
 'top_down_pyramid_size': 256,
 'rpn_anchor_scales': (32, 64, 128, 256, 512),
 'rpn_anchor_ratios': [0.5, 1, 2],
 'rpn_anchor_stride': 1,
 'rpn_train_

In [6]:
model = mask_rcnn_functional(config=CONFIG)

[MaskRCNN] Training mode

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[MaskRCNN] Backbone architecture: mobilenet


/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


[MaskRCNN] Total params: 24,073,932
[MaskRCNN] Trainable params: 23,755,980


In [7]:
CONFIG['training']

True

In [8]:
CONFIG['backbone']

'mobilenet'

In [10]:
base_dir = os.getcwd().replace('src', 'balloon')
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

train_dataset = balloon.BalloonDataset(images_dir=train_dir,
                                       class_key='object',
                                       classes_dict=CONFIG['class_dict'],
                                       preprocess_transform=preprocess.get_input_preprocess(
                                           normalize=CONFIG['normalization']
                                       ),
                                       augmentation=aug.get_training_augmentation(),
                                       json_annotation_key=None,
                                       **CONFIG
                                       )

val_dataset = balloon.BalloonDataset(images_dir=val_dir,
                                     class_key='object',
                                     classes_dict=CONFIG['class_dict'],
                                     preprocess_transform=preprocess.get_input_preprocess(
                                         normalize=CONFIG['normalization']
                                     ),
                                     json_annotation_key=None,
                                     **CONFIG
                                     )

Found annotation file: via_region_data.json in dataset path: /home/alexander/Documents/py_projects/github/maskrcnn_tf2/balloon/train
Found annotation file: via_region_data.json in dataset path: /home/alexander/Documents/py_projects/github/maskrcnn_tf2/balloon/val


In [11]:
train_model(model, 
            train_dataset=train_dataset,
            val_dataset=val_dataset,
            config=CONFIG, 
            weights_path=None)

train DataLoader. Steps per epoch: 61
val DataLoader. Steps per epoch: 13
MaskRCNN Losses:
rpn_class_loss: <layers.losses.RPNClassLoss object at 0x7fe74c085a90>
rpn_bbox_loss: <layers.losses.RPNBboxLoss object at 0x7fe74c0850d0>
mrcnn_class_loss: <layers.losses.MRCNNClassLoss object at 0x7fe74c085290>
mrcnn_bbox_loss: <layers.losses.MRCNNBboxLoss object at 0x7fe74c085cd0>
mrcnn_mask_loss: <layers.losses.MRCNNMaskLoss object at 0x7fe7287ec5d0>
l2_regularizer: <layers.losses.L2RegLoss object at 0x7fe74c130510>

Epoch 1/30
Instructions for updating:
Use tf.identity instead.
 7/61 [==>...........................] - ETA: 47s - rpn_class_loss: 0.8794 - rpn_bbox_loss: 6.6544 - mrcnn_class_loss: 1.0494 - mrcnn_bbox_loss: 8.2585 - mrcnn_mask_loss: 0.6689 - l2_regularizer: 0.0084 - loss_sum: 17.5189

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

